In [2]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import holidays
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score,classification_report,confusion_matrix
import matplotlib.pyplot as plt
from scipy.stats import  randint
import math
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import pickle

/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


## Build X_train

In [4]:
X_train = pd.read_csv('fl01_train')

In [3]:
X_train.info()

NameError: name 'X_train' is not defined

In [5]:
X_train = X_train[['fl_date','op_unique_carrier','op_carrier_fl_num','origin_airport_id','crs_dep_time','dest_airport_id','crs_arr_time','distance','arr_delay','carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']]

In [27]:
X_train.head()

,distance,fl_carrier,fl_weather,fl_nas,fl_security,fl_late_aircraft,dep_carrier,arr_carrier,dep_weather,arr_weather,dep_nas,arr_nas,dep_security,arr_security,dep_late_aircraft,arr_late_aircraft,is_holiday,dow_sin,dow_cos
0,2106.0,0.250000,0.000000,31.25,0.0,0.250000,13.813187,10.866667,1.175824,0.000000,7.604396,17.200000,0.000000,1.933333,27.538462,10.866667,0,-0.974927,-0.222525
1,1243.0,14.500000,0.000000,13.50,0.0,15.500000,32.384615,13.682540,0.000000,0.000000,42.730769,5.206349,0.000000,0.000000,38.115385,13.682540,0,-0.781834,0.623486
2,612.0,7.666667,10.666667,14.00,0.0,15.333333,19.782065,16.755287,5.087591,1.432024,16.297185,14.552870,0.004171,0.021148,31.207508,16.755287,0,-0.000005,1.000000
3,515.0,54.600000,0.000000,14.00,0.0,0.000000,25.914530,14.190608,0.350427,3.511050,29.418803,14.882597,0.000000,0.030387,49.871795,14.190608,0,-0.974927,-0.222525
4,1561.0,3.000000,0.000000,19.00,0.0,0.000000,26.173913,13.481481,5.478261,2.629630,18.521739,29.962963,0.000000,0.000000,0.000000,13.481481,0,-0.000005,1.000000


In [6]:
train_y = pd.read_csv('train_y_del_cls')

In [7]:
X_train['delay_class'] = train_y['delay_class']

In [6]:
X_train.isnull().sum()

fl_date                     0
op_unique_carrier           0
op_carrier_fl_num           0
origin_airport_id           0
crs_dep_time                0
dest_airport_id             0
crs_arr_time                0
distance                    0
arr_delay               17854
carrier_delay          812276
weather_delay          812276
nas_delay              812276
security_delay         812276
late_aircraft_delay    812276
delay_class                 0
dtype: int64

In [38]:
## Combine 5 delays into one delay_class
def delay_cla(a,b,c,d,e):
    l = pd.Series([a,b,c,d,e]).fillna(0)
    if l.sum() == 0:
        return 0
    else: 
        return l.idxmax() + 1

#print(delay_cla(None,1,None,3,None))
X_train['delay_class'] = X_train[['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']].apply(lambda X_train: delay_cla(X_train['carrier_delay'],X_train['weather_delay'],X_train['nas_delay'],X_train['security_delay'],X_train['late_aircraft_delay']),axis=1)
#X_train['delay_class'] = np.vectorize(delay_cla)(X_train[['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']])
#X_train['delay_class'] = np.vectorize(delay_cla)(X_train['carrier_delay'],X_train['weather_delay'],X_train['nas_delay'],X_train['security_delay'],X_train['late_aircraft_delay'])

In [8]:
#combine features make uni_fl
def combine_str_num(x,y):
    return x + str(y)
X_train['uni_fl'] = np.vectorize(combine_str_num)(X_train['op_unique_carrier'],X_train['op_carrier_fl_num'])
X_train.drop(columns=['op_unique_carrier','op_carrier_fl_num'],inplace=True)


In [9]:
# feature coding for flightno --- delay
unifl_carrier_mean = X_train.groupby(['uni_fl'])['carrier_delay'].mean()
X_train['fl_carrier'] =X_train['uni_fl'].map(unifl_carrier_mean)
X_train['fl_carrier'].fillna(0,inplace=True)

unifl_weather_mean = X_train.groupby(['uni_fl'])['weather_delay'].mean()
X_train['fl_weather'] =X_train['uni_fl'].map(unifl_weather_mean)
X_train['fl_weather'].fillna(0,inplace=True)

unifl_nas_mean = X_train.groupby(['uni_fl'])['nas_delay'].mean()
X_train['fl_nas'] =X_train['uni_fl'].map(unifl_nas_mean)
X_train['fl_nas'].fillna(0,inplace=True)

unifl_security_mean = X_train.groupby(['uni_fl'])['security_delay'].mean()
X_train['fl_security'] =X_train['uni_fl'].map(unifl_security_mean)
X_train['fl_security'].fillna(0,inplace=True)

unifl_late_aircraft_mean = X_train.groupby(['uni_fl'])['late_aircraft_delay'].mean()
X_train['fl_late_aircraft'] =X_train['uni_fl'].map(unifl_late_aircraft_mean)
X_train['fl_late_aircraft'].fillna(0,inplace=True)

In [10]:
## Combine the airportid and related hour
def airport_hour(id,t):
    '''
    combine airport and hour to a new field
    '''
    h = str(t)[:-2]
    if len(h) < 1:
        return str(id) + '24'
    elif len(h) < 2:
        return str(id) + '0' + h
    else:
        return str(id) + h
    
X_train['arr_airport_hour'] = np.vectorize(airport_hour)(X_train['dest_airport_id'],X_train['crs_arr_time'])
X_train['dep_airport_hour'] = np.vectorize(airport_hour)(X_train['origin_airport_id'],X_train['crs_dep_time'])

X_train.drop(columns=['dest_airport_id','crs_arr_time','origin_airport_id','crs_dep_time'],inplace=True)

In [11]:
# feature coding airport_hour to delays
dep_hour_carrier_mean = X_train.groupby(['dep_airport_hour'])['carrier_delay'].mean()
X_train['dep_carrier'] =X_train['dep_airport_hour'].map(dep_hour_carrier_mean)
arr_hour_carrier_mean = X_train.groupby(['arr_airport_hour'])['carrier_delay'].mean()
X_train['arr_carrier'] =X_train['arr_airport_hour'].map(arr_hour_carrier_mean)
X_train['dep_carrier'].fillna(0,inplace=True)
X_train['arr_carrier'].fillna(0,inplace=True)

dep_hour_weather_mean = X_train.groupby(['dep_airport_hour'])['weather_delay'].mean()
X_train['dep_weather'] =X_train['dep_airport_hour'].map(dep_hour_weather_mean)
arr_hour_weather_mean = X_train.groupby(['arr_airport_hour'])['weather_delay'].mean()
X_train['arr_weather'] =X_train['arr_airport_hour'].map(arr_hour_weather_mean)
X_train['dep_weather'].fillna(0,inplace=True)
X_train['arr_weather'].fillna(0,inplace=True)

dep_hour_nas_mean = X_train.groupby(['dep_airport_hour'])['nas_delay'].mean()
X_train['dep_nas'] =X_train['dep_airport_hour'].map(dep_hour_nas_mean)
arr_hour_nas_mean = X_train.groupby(['arr_airport_hour'])['nas_delay'].mean()
X_train['arr_nas'] =X_train['arr_airport_hour'].map(arr_hour_nas_mean)
X_train['dep_nas'].fillna(0,inplace=True)
X_train['arr_nas'].fillna(0,inplace=True)

dep_hour_security_mean = X_train.groupby(['dep_airport_hour'])['security_delay'].mean()
X_train['dep_security'] =X_train['dep_airport_hour'].map(dep_hour_security_mean)
arr_hour_security_mean = X_train.groupby(['arr_airport_hour'])['security_delay'].mean()
X_train['arr_security'] =X_train['arr_airport_hour'].map(arr_hour_security_mean)
X_train['dep_security'].fillna(0,inplace=True)
X_train['arr_security'].fillna(0,inplace=True)

dep_hour_late_aircraft_mean = X_train.groupby(['dep_airport_hour'])['late_aircraft_delay'].mean()
X_train['dep_late_aircraft'] =X_train['dep_airport_hour'].map(dep_hour_late_aircraft_mean)
arr_hour_late_aircraft_mean = X_train.groupby(['arr_airport_hour'])['carrier_delay'].mean()
X_train['arr_late_aircraft'] =X_train['arr_airport_hour'].map(arr_hour_late_aircraft_mean)
X_train['dep_late_aircraft'].fillna(0,inplace=True)
X_train['arr_late_aircraft'].fillna(0,inplace=True)

In [12]:
#transform fl_date
#def getmonth(x):
#    return x.split(sep='-')[1]
#X_train['month'] = X_train['fl_date'].apply(getmonth)

def getdayofweek(x):
    year_s, mon_s, day_s = x.split('-')
    fl_d = datetime(int(year_s), int(mon_s), int(day_s))
    return fl_d.weekday() + 1
X_train['day_of_week'] = X_train['fl_date'].apply(getdayofweek)

us_holidays = holidays.UnitedStates()
def isholiday(x):
    year_s, mon_s, day_s = x.split('-')
    if datetime(int(year_s), int(mon_s), int(day_s)) in us_holidays:
        return 1
    else:
        return 0
X_train['is_holiday'] = X_train['fl_date'].apply(isholiday)


pi=3.14159
def transformation(column):
  max_value = column.max()
  sin_values = [math.sin((2*pi*x)/max_value) for x in list(column)]
  cos_values = [math.cos((2*pi*x)/max_value) for x in list(column)]
  return sin_values, cos_values

sin_l, cos_l = transformation(X_train['day_of_week'])

X_train['dow_sin'] = sin_l
X_train['dow_cos'] = cos_l

In [13]:
y_train = X_train['delay_class']
X_train.drop(columns=['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay','fl_date','arr_delay','day_of_week','delay_class','uni_fl','dep_airport_hour','arr_airport_hour'],inplace=True)

#X_train = pd.get_dummies(X_train,columns=['month','day_of_week'])

In [18]:
#scaler = StandardScaler()
#X = scaler.fit_transform(X_train)

In [ ]:
#x_small = X[:100000]
#y_small = y_train[:100000]

### Build X_test

In [14]:
X_test = pd.read_csv('fl01_test')

In [15]:
X_test = X_test[['fl_date','op_unique_carrier','op_carrier_fl_num','origin_airport_id','crs_dep_time','dest_airport_id','crs_arr_time','distance','arr_delay','carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']]

In [16]:
X_test.head()

,fl_date,op_unique_carrier,op_carrier_fl_num,origin_airport_id,crs_dep_time,dest_airport_id,crs_arr_time,distance,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018-01-21,AA,9,12478,700,14771,1049,2586.0,-36.0,NaN,NaN,NaN,NaN,NaN
1,2018-01-12,WN,1832,11278,600,13204,825,759.0,82.0,81.0,0.0,1.0,0.0,0.0
2,2019-01-27,WN,1699,14893,825,10423,1345,1481.0,-24.0,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,EV,3978,13930,715,14524,1018,642.0,6.0,NaN,NaN,NaN,NaN,NaN
4,2018-01-22,F9,649,12953,920,11193,1133,585.0,19.0,0.0,0.0,19.0,0.0,0.0


In [71]:
X_test.isnull().sum()

distance                    0
carrier_delay          202740
weather_delay          202740
nas_delay              202740
security_delay         202740
late_aircraft_delay    202740
uni_fl                      0
arr_airport_hour            0
dep_airport_hour            0
is_holiday                  0
dow_sin                     0
dow_cos                     0
dtype: int64

In [43]:
def delay_cla(a,b,c,d,e):
    l = pd.Series([a,b,c,d,e]).fillna(0)
    if l.sum() == 0:
        return 0
    else: 
        return l.idxmax() + 1
X_test['delay_class'] = np.vectorize(delay_cla)(X_test['carrier_delay'],X_test['weather_delay'],X_test['nas_delay'],X_test['security_delay'],X_test['late_aircraft_delay'])

In [17]:
test_y = pd.read_csv('test_y_del_cls')
X_test['delay_class'] = test_y['delay_class']

In [18]:
#combine features create uni flight no
def combine_str_num(x,y):
    return x + str(y)
X_test['uni_fl'] = np.vectorize(combine_str_num)(X_test['op_unique_carrier'],X_test['op_carrier_fl_num'])
X_test.drop(columns=['op_unique_carrier','op_carrier_fl_num'],inplace=True)

In [19]:
# feature coding for flightno --- delay
unifl_carrier_mean = X_test.groupby(['uni_fl'])['carrier_delay'].mean()
X_test['fl_carrier'] =X_test['uni_fl'].map(unifl_carrier_mean)
X_test['fl_carrier'].fillna(0,inplace=True)

unifl_weather_mean = X_test.groupby(['uni_fl'])['weather_delay'].mean()
X_test['fl_weather'] =X_test['uni_fl'].map(unifl_weather_mean)
X_test['fl_weather'].fillna(0,inplace=True)

unifl_nas_mean = X_test.groupby(['uni_fl'])['nas_delay'].mean()
X_test['fl_nas'] =X_test['uni_fl'].map(unifl_nas_mean)
X_test['fl_nas'].fillna(0,inplace=True)

unifl_security_mean = X_test.groupby(['uni_fl'])['security_delay'].mean()
X_test['fl_security'] =X_test['uni_fl'].map(unifl_security_mean)
X_test['fl_security'].fillna(0,inplace=True)

unifl_late_aircraft_mean = X_test.groupby(['uni_fl'])['late_aircraft_delay'].mean()
X_test['fl_late_aircraft'] =X_test['uni_fl'].map(unifl_late_aircraft_mean)
X_test['fl_late_aircraft'].fillna(0,inplace=True)

In [20]:
## Combine the airportid and related hour
def airport_hour(id,t):
    '''
    combine airport and hour to a new field
    '''
    h = str(t)[:-2]
    if len(h) < 1:
        return str(id) + '24'
    elif len(h) < 2:
        return str(id) + '0' + h
    else:
        return str(id) + h
    
X_test['arr_airport_hour'] = np.vectorize(airport_hour)(X_test['dest_airport_id'],X_test['crs_arr_time'])
X_test['dep_airport_hour'] = np.vectorize(airport_hour)(X_test['origin_airport_id'],X_test['crs_dep_time'])

X_test.drop(columns=['dest_airport_id','crs_arr_time','origin_airport_id','crs_dep_time'],inplace=True)

In [21]:
# feature coding airport_hour to delays
dep_hour_carrier_mean = X_test.groupby(['dep_airport_hour'])['carrier_delay'].mean()
X_test['dep_carrier'] =X_test['dep_airport_hour'].map(dep_hour_carrier_mean)
arr_hour_carrier_mean = X_test.groupby(['arr_airport_hour'])['carrier_delay'].mean()
X_test['arr_carrier'] =X_test['arr_airport_hour'].map(arr_hour_carrier_mean)
X_test['dep_carrier'].fillna(0,inplace=True)
X_test['arr_carrier'].fillna(0,inplace=True)

dep_hour_weather_mean = X_test.groupby(['dep_airport_hour'])['weather_delay'].mean()
X_test['dep_weather'] =X_test['dep_airport_hour'].map(dep_hour_weather_mean)
arr_hour_weather_mean = X_test.groupby(['arr_airport_hour'])['weather_delay'].mean()
X_test['arr_weather'] =X_test['arr_airport_hour'].map(arr_hour_weather_mean)
X_test['dep_weather'].fillna(0,inplace=True)
X_test['arr_weather'].fillna(0,inplace=True)

dep_hour_nas_mean = X_test.groupby(['dep_airport_hour'])['nas_delay'].mean()
X_test['dep_nas'] =X_test['dep_airport_hour'].map(dep_hour_nas_mean)
arr_hour_nas_mean = X_test.groupby(['arr_airport_hour'])['nas_delay'].mean()
X_test['arr_nas'] =X_test['arr_airport_hour'].map(arr_hour_nas_mean)
X_test['dep_nas'].fillna(0,inplace=True)
X_test['arr_nas'].fillna(0,inplace=True)

dep_hour_security_mean = X_test.groupby(['dep_airport_hour'])['security_delay'].mean()
X_test['dep_security'] =X_test['dep_airport_hour'].map(dep_hour_security_mean)
arr_hour_security_mean = X_test.groupby(['arr_airport_hour'])['security_delay'].mean()
X_test['arr_security'] =X_test['arr_airport_hour'].map(arr_hour_security_mean)
X_test['dep_security'].fillna(0,inplace=True)
X_test['arr_security'].fillna(0,inplace=True)

dep_hour_late_aircraft_mean = X_test.groupby(['dep_airport_hour'])['late_aircraft_delay'].mean()
X_test['dep_late_aircraft'] =X_test['dep_airport_hour'].map(dep_hour_late_aircraft_mean)
arr_hour_late_aircraft_mean = X_test.groupby(['arr_airport_hour'])['carrier_delay'].mean()
X_test['arr_late_aircraft'] =X_test['arr_airport_hour'].map(arr_hour_late_aircraft_mean)
X_test['dep_late_aircraft'].fillna(0,inplace=True)
X_test['arr_late_aircraft'].fillna(0,inplace=True)

In [22]:
#transform fl_date
#def getmonth(x):
#    return x.split(sep='-')[1]
#X_test['month'] = X_test['fl_date'].apply(getmonth)

def getdayofweek(x):
    year_s, mon_s, day_s = x.split('-')
    fl_d = datetime(int(year_s), int(mon_s), int(day_s))
    return fl_d.weekday() + 1
X_test['day_of_week'] = X_test['fl_date'].apply(getdayofweek)

us_holidays = holidays.UnitedStates()
def isholiday(x):
    year_s, mon_s, day_s = x.split('-')
    if datetime(int(year_s), int(mon_s), int(day_s)) in us_holidays:
        return 1
    else:
        return 0
X_test['is_holiday'] = X_test['fl_date'].apply(isholiday)



pi=3.14159
def transformation(column):
  max_value = column.max()
  sin_values = [math.sin((2*pi*x)/max_value) for x in list(column)]
  cos_values = [math.cos((2*pi*x)/max_value) for x in list(column)]
  return sin_values, cos_values

sin_l, cos_l = transformation(X_test['day_of_week'])

X_test['dow_sin'] = sin_l
X_test['dow_cos'] = cos_l



In [23]:
y_test = X_test['delay_class']

X_test = X_test.drop(columns=['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay','fl_date','arr_delay','day_of_week','delay_class','uni_fl','dep_airport_hour','arr_airport_hour'])

#X_test = pd.get_dummies(X_test,columns=['month','day_of_week'])

#X_test = scaler.transform(X_test)

In [23]:
###  Bayes

In [31]:
gnb = GaussianNB()
gnb.fit(X_train,y_train)
y_pred = gnb.predict(X_test)

In [33]:
y_pred = gnb.predict(X_test)

In [34]:
unique, counts = np.unique(y_pred, return_counts=True)
print(unique,counts)

[0 1 2 3 4 5] [ 16521  41683   8672  76576   1244 103278]


In [35]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",accuracy_score(y_test, y_pred))
#print("ROC score", roc_auc_score(y_test,y_pred,multi_class='ovr'))

Accuracy: 0.1549073693209772


In [78]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.86      0.84    202740
           1       0.12      0.10      0.11     12054
           2       0.07      0.05      0.06      1756
           3       0.15      0.11      0.13     14254
           4       0.09      0.17      0.11        66
           5       0.17      0.14      0.16     17104

    accuracy                           0.73    247974
   macro avg       0.24      0.24      0.24    247974
weighted avg       0.70      0.73      0.71    247974



In [72]:
y_pred_pro = gnb.predict_proba(X_test)

In [47]:
gnb.class_prior_

In [52]:
y_pred_pro[:3]

array([0.20215096, 0.26113699, 0.21012088, 0.21025372, 0.00632843,
       0.05484196, 0.05516707])

In [41]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",accuracy_score(y_test, y_pred_r))

Accuracy: 0.8176220087589828


In [42]:
print(classification_report(y_test,y_pred_r))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90    202740
           1       0.00      0.00      0.00     12054
           2       0.00      0.00      0.00      1756
           3       0.85      0.00      0.00     14254
           4       0.00      0.00      0.00        66
           5       0.00      0.00      0.00     17104

    accuracy                           0.82    247974
   macro avg       0.28      0.17      0.15    247974
weighted avg       0.72      0.82      0.74    247974



/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
confusion_matrix(y_test,y_pred_r)

array([[202738,      0,      0,      2,      0,      0],
       [ 12054,      0,      0,      0,      0,      0],
       [  1756,      0,      0,      0,      0,      0],
       [ 14243,      0,      0,     11,      0,      0],
       [    66,      0,      0,      0,      0,      0],
       [ 17104,      0,      0,      0,      0,      0]])

In [105]:
pickle.dump(rg,open('Grid_RandomForest_1.sav','wb'))

In [30]:
y_pred = loaded_model.predict(X_test)

In [54]:
print("Accuracy:",accuracy_score(y_test, y_pred_r))

Accuracy: 0.4876559639317025


In [62]:
y_pred_r

array([1, 2, 2, ..., 2, 2, 2])

In [56]:
print(unique,counts)

[0 1 2 3 4 5 6] [    62  16688 180105  30318  12613   6798   1390]


In [55]:
unique, counts = np.unique(y_pred_r, return_counts=True)

In [60]:
y_arr = np.array([arr_d_match[i] for i in y_pred_r])

In [61]:
y_arr

array([-31.1476422 , -11.23198414, -11.23198414, ..., -11.23198414,
       -11.23198414, -11.23198414])

In [63]:
arr_d_match

delay_cls
0    -84.188470
1    -31.147642
2    -11.231984
3      8.376291
4     35.723876
5    100.855927
6    320.689599
Name: arr_delay, dtype: float64

In [67]:
y_arr = np.array([arr_d_match[i] for i in y_pred])
r2_score(Y_compare,y_arr)

-0.09276152459671638

In [64]:
## four round  48.09246
r2_score(y_test,y_pred)

0.025932099879509285

In [94]:
## third round  48.67
r2_score(y_test,y_pred)

0.03500066824855497

In [21]:
## second round with arr_delay <480
#r2_score(y_test, y_pred)

0.02144653066658464

In [27]:
## first round with arr_delay < 180
#

0.013895135285908466

In [ ]:
## first try on XGBoost 
#r2_score(y_test,y_pred)

-0.0005833909885790689